In [ ]:
# Scraping 
# https://www.skiresort.info/ski-resorts/usa/

In [22]:
# Import dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint

In [23]:
request = requests.get("https://www.skiresort.info/ski-resorts/usa/")

In [24]:
prep_soup = bs(request.text, 'html.parser')

In [58]:
panel = prep_soup.find('div', class_= "panel panel-primary")

In [59]:
# pprint(panel)

In [60]:
pages = prep_soup.find('ul', class_= "pagination pull-right")

In [61]:
links_with_text = []
for a in pages.find_all('a', href=True): 
    if a.text: 
        links_with_text.append(a['href'])

In [62]:
links_with_text

['https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/3/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/']

In [71]:
# name = panel.find_all("div", class_= "panel-body middle-padding").find("div", class_="col-sm-11 col-xs-10")

In [86]:
name = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

In [101]:
resort = []
for x in name:
    lists = x.find_all("a", class_="h3")
#     print("-------------------------------------------")
    resort.append(lists)
#     resort_name.append(resort.text)

In [102]:
resort

[[<a class="h3" href="https://www.skiresort.info/ski-resort/vail/"> Vail </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/telluride/"> Telluride </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/beaver-creek/"> Beaver Creek </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/mammoth-mountain/"> Mammoth Mountain </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/snowmass/"> Snowmass </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/breckenridge/"> Breckenridge </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/keystone/"> Keystone </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/sunday-river/"> Sunday River </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/killington/"> Killington </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/heavenly/"> Heavenly </a>],
 [<a class="h3" href="https://www.skiresort.info/ski-resort/winter-park-resort/"> 

In [108]:
resort[0][0].text

' Vail '

In [111]:
resort_names = []
resort_links = []
for x in range(len(resort)):
    resort_names.append(resort[x][0].text)
    resort_links.append(resort[x][0]["href"])

In [112]:
# resort_names
resort_links

['https://www.skiresort.info/ski-resort/vail/',
 'https://www.skiresort.info/ski-resort/telluride/',
 'https://www.skiresort.info/ski-resort/beaver-creek/',
 'https://www.skiresort.info/ski-resort/mammoth-mountain/',
 'https://www.skiresort.info/ski-resort/snowmass/',
 'https://www.skiresort.info/ski-resort/breckenridge/',
 'https://www.skiresort.info/ski-resort/keystone/',
 'https://www.skiresort.info/ski-resort/sunday-river/',
 'https://www.skiresort.info/ski-resort/killington/',
 'https://www.skiresort.info/ski-resort/heavenly/',
 'https://www.skiresort.info/ski-resort/winter-park-resort/',
 'https://www.skiresort.info/ski-resort/stowe/',
 'https://www.skiresort.info/ski-resort/squaw-valley/',
 'https://www.skiresort.info/ski-resort/steamboat/',
 'https://www.skiresort.info/ski-resort/park-city/',
 'https://www.skiresort.info/ski-resort/big-sky-resort/',
 'https://www.skiresort.info/ski-resort/aspen-highlands/',
 'https://www.skiresort.info/ski-resort/copper-mountain/',
 'https://ww

In [ ]:
# .find_all("a", class_="h3")